<img src="../thumbnail.svg" width=250 alt="Intake Logo"></img>

# Creating Intake Catalogs

---

## Overview
In the last lesson we learned to use Intake catalogs to simplify the process of accessing research data. In this lesson we will walk through the steps of creating a catalog for your research data by recreating the catalog in the previous lesson.

1. Creating an Intake catalog
1. Documenting your data source
1. Share your catalog on Github.

## Prerequisites


| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Intake]() | Necessary | |
| [Understanding of yaml](https://yaml.org/spec/1.2.2/#chapter-1-introduction-to-yaml) | Necessary |  |
| [Getting Started with Github](https://foundations.projectpythia.org/foundations/getting-started-github.html) | Necessary | |
| [Intro to Pandas](https://foundations.projectpythia.org/core/pandas/pandas.html) | Helpful | |


- **Time to learn**: 45 minutes

---

## Imports

In [1]:
import intake
import intake_xarray
import intake_markdown
import requests
import aiohttp
import s3fs
import yaml
import json
import datetime
import os

## Setting up the environment

By the end of this tutorial we will have created a git repository that we can host on Github to share our catalog. 

Start by [creating a Github repository](https://docs.github.com/en/get-started/quickstart/create-a-repo) called "intake_demo", and then clone the repository to your local machine. 

```bash
git clone {path to Github repository}
```

Intake catalogs can be a simple yaml file. We can create the yaml file programmatically by converting nested python dictionaries to yaml. A Intake catalog has two main parts metadata and sources. The metadata can be arbitrary with a few exceptions. The sources section is a mapping between a data source name and its properties. For more information about Intake catalogs, see Intake's [documentation](https://intake.readthedocs.io/en/latest/catalog.html)

In [2]:
description = "Catalog containing Mesowest's HRRR data. See readme source for more information."

catalog = {'metadata': {'version': 1,
                       'description': description},
           'sources': {}}

os.makedirs("intake_demo", exist_ok=True) #only needed for building this notebook
with open('intake_demo/catalog.yml', 'w') as f:
    yaml.dump(catalog, f)

You will now notice a new file in your `intake_demo` directory called `catalog.yml` with the following contents.

In [3]:
with open('intake_demo/catalog.yml', 'r') as f:
    print(f.read())

metadata:
  description: Catalog containing Mesowest's HRRR data. See readme source for more
    information.
  version: 1
sources: {}



## Adding Your First Data Source
Intake only knows how to handle a few different data formats. To handle other formats it uses plugable drivers. To use the [Mesowest's HRRR Zarr data](https://mesowest.utah.edu/html/hrrr/) we will use the intake-xarray package which provides a driver for reading Zarr data into [Xarray](https://foundations.projectpythia.org/core/xarray.html) datasets. Drivers are installed as python packages and integrate into the Intake library. When intalled Intake creates a open_{driver} method for each driver in the package. Installing the intake-xarray package allows us to access zarr data using the `open_zarr` method.

Mesowest's HRRR Zarr data is stored in AWS. The [file structure](https://mesowest.utah.edu/html/hrrr/zarr_documentation/html/zarr_variables.html) of the [hrrrzarr S3 bucket](https://hrrrzarr.s3.amazonaws.com/index.html) looks like 

s3://hrrrzarr/sfc/{yyyymmdd}/{yyyymmdd}_{hh}z_anl.zarr/{level}/{param}/{level}

where

- yyyy = four digit year
- mm = two digit month
- dd = two digit day of month
- hh = two digit hour of the day
- level = level of atmoshpere the data describes
- param = the parameter your interested in

To load a complete dataset we need the Zarr arrays from two urls

s3://hrrrzarr/sfc/{yyyymmdd}/{yyyymmdd}_{hh}z_anl.zarr/{level}/{parameter}{level}

s3://hrrrzarr/sfc/{yyyymmdd}/{yyyymmdd}_{hh}z_anl.zarr/{level}/{parameter}


Lets load surface temperature data from August 24, 2016

In [4]:
urls = ['s3://hrrrzarr/sfc/20160824/20160824_00z_anl.zarr/surface/TMP/surface',
           's3://hrrrzarr/sfc/20160824/20160824_00z_anl.zarr/surface/TMP']

chunks = {'projection_x_coordinate': 1799,
          'projection_y_coordinate': 1059}

so = {"anon": True}

source = intake.open_zarr(urls, chunks=chunks, storage_options=so)
source.name = 'hrrrzarr'
source.description = "Mesowest's HRRR data. See readme source for more information."
ds = source.read()
ds

<xarray.Dataset>
Dimensions:                  (projection_x_coordinate: 1799,
                              projection_y_coordinate: 1059)
Coordinates:
  * projection_x_coordinate  (projection_x_coordinate) float64 -2.698e+06 ......
  * projection_y_coordinate  (projection_y_coordinate) float64 -1.587e+06 ......
Data variables:
    TMP                      (projection_y_coordinate, projection_x_coordinate) float16 ...
    forecast_period          timedelta64[ns] 00:00:00
    forecast_reference_time  datetime64[ns] 2016-08-24
    height                   float64 1e+03
    pressure                 float64 2.5e+04
    time                     datetime64[ns] 2016-08-24

Above we used the `storage_options` argument to tell Intake how to access data on AWS. In this case we accessed the data as an anonymous user.

When you use Intake's `open_{driver}` function, it creates a catalog entry for the source. You can view the yaml using the source's `yaml` function.

In [5]:
print(source.yaml())

sources:
  hrrrzarr:
    args:
      chunks:
        projection_x_coordinate: 1799
        projection_y_coordinate: 1059
      storage_options:
        anon: true
      urlpath:
      - s3://hrrrzarr/sfc/20160824/20160824_00z_anl.zarr/surface/TMP/surface
      - s3://hrrrzarr/sfc/20160824/20160824_00z_anl.zarr/surface/TMP
    description: Mesowest's HRRR data. See readme source for more information.
    driver: intake_xarray.xzarr.ZarrSource
    metadata:
      coords: !!python/tuple
      - projection_x_coordinate
      - projection_y_coordinate
      data_vars:
        TMP:
        - projection_x_coordinate
        - projection_y_coordinate
        forecast_period: []
        forecast_reference_time: []
        height: []
        pressure: []
        time: []
      dims:
        projection_x_coordinate: 1799
        projection_y_coordinate: 1059



### Modifying the Source

If we wanted we could add this yaml to our catalog and we could then load this data using Intake. However, there are many datasets we can load from the Zarr store with almost the same catalog entry. Making a separate entry for each would make the catalog cluttered and harder to use. Instead we will generalize this catalog entry so it applies to many datasets. Then we will create user parameters to give the catalog user the abillity to select the data they want.

To generalize the source we need Intake to dynamically generate urls pointing to the Zarr arrays based off user set parameters. We will take the source created by the `open_zarr` function convert it to a python dictionary and then modify it to include user parameters. We can then use those parameters to generate the urls. Intake provides Jinja templating in catalogs to make this simple. Let's start by defining user parameters.

In [6]:
source_dict = yaml.load(source.yaml(), Loader=yaml.CLoader)

parameters = {}
parameters['level'] = {'description': "Parameter specifying level in the atmosphere. Corresponds to 'Vertical Level' column in data_dictionary",
                       'type': 'str',
                       'default': 'surface'}

parameters['param'] = {'description': "Specifies what parameter your dataset will contain. Corresponds to 'Parameter Short Name' in data_dictionary",
                       'type': 'str',
                       'default': 'TMP'}

parameters['date'] = {'description': "Date and hour of data.",
                      'type': 'datetime',
                      'default': "2016-08-24T00:00:00",
                      'min': "2016-08-24T00:00:00"}



sources = source_dict['sources']
hrrr_zarr = sources['hrrrzarr']
hrrr_zarr['parameters'] = parameters

With the parameters defined we can now use them to create the urls using Jinja syntax.

In [7]:
urls = ["s3://hrrrzarr/sfc/{{date.strftime('%Y%m%d/%Y%m%d_%Hz_anl.zarr')}}/{{level}}/{{param}}",
        "s3://hrrrzarr/sfc/{{date.strftime('%Y%m%d/%Y%m%d_%Hz_anl.zarr')}}/{{level}}/{{param}}/{{level}}"]

hrrr_zarr['args']['urlpath'] = urls

Now that we have a more generalized source, some of the metadata is too specific. To fix this we will just remove the `data_var`section from the source's metadata.

In [8]:
hrrr_zarr['metadata'].pop('data_vars', None)
print(yaml.dump(source_dict))

sources:
  hrrrzarr:
    args:
      chunks:
        projection_x_coordinate: 1799
        projection_y_coordinate: 1059
      storage_options:
        anon: true
      urlpath:
      - s3://hrrrzarr/sfc/{{date.strftime('%Y%m%d/%Y%m%d_%Hz_anl.zarr')}}/{{level}}/{{param}}
      - s3://hrrrzarr/sfc/{{date.strftime('%Y%m%d/%Y%m%d_%Hz_anl.zarr')}}/{{level}}/{{param}}/{{level}}
    description: Mesowest's HRRR data. See readme source for more information.
    driver: intake_xarray.xzarr.ZarrSource
    metadata:
      coords: !!python/tuple
      - projection_x_coordinate
      - projection_y_coordinate
      dims:
        projection_x_coordinate: 1799
        projection_y_coordinate: 1059
    parameters:
      date:
        default: '2016-08-24T00:00:00'
        description: Date and hour of data.
        min: '2016-08-24T00:00:00'
        type: datetime
      level:
        default: surface
        description: Parameter specifying level in the atmosphere. Corresponds to
          'Vertica

## Documenting the Data

We have created a data source, but it may be a little tricky to use. We need a way to let users know what options they have for the `level` and `param` user parameters we defined earlier. The inventory.csv file, created from [Mesowest's HRRR Zarr Variable List](https://mesowest.utah.edu/html/hrrr/zarr_documentation/html/zarr_variables.html), in this directory contains a table which shows what parameters are in the Zarr store and at what level in the atmosphere those parameters are available. Let's open it as a source and add it to our source dictionary.

In [9]:
source = intake.open_csv('inventory.csv',
                                  csv_kwargs={'keep_default_na': False})
source.name = 'data_dictionary'
source.description = 'Describes the data in the hrrrzarr source'

source.read()

,Parameter Long Name,Vertical Level,Parameter Short Name,Units,1st Version Available,Analysis or Forcast,Notes
0,Hail,0.1_sigma_layer,HAIL_max_fcst,m,V4,anl,
1,Hail,0.1_sigma_layer,HAIL_1hr_max_fcst,m,V4,fcst,
2,Vertical Velocity,0.5_0.8_sigma_layer,DZDT_ave_fcst,m/s,V3,anl,
3,Vertical Velocity,0.5_0.8_sigma_layer,DZDT_1hr_ave_fcst,m/s,V3,fcst,
4,U-Component Vertical Wind Shear,0_1000m_above_ground,VUCSH,1/s,V3,both,
...,...,...,...,...,...,...,...
187,Upward Short-Wave Rad. Flux,top_of_atmosphere,USWRF,W/m2,V3,both,
188,"Simulated Brightness Temperature for GOES 11, ...",top_of_atmosphere,SBT113,K,V3,both,
189,"Simulated Brightness Temperature for GOES 11, ...",top_of_atmosphere,SBT114,K,V3,both,
190,"Simulated Brightness Temperature for GOES 12, ...",top_of_atmosphere,SBT123,K,V3,both,


The `Vertical Level` column corresponds to the level paremeter in our data source and the `Parameter Short Name` corresponds to the `param` parameter.

In [10]:
source

data_dictionary:
  args:
    csv_kwargs:
      keep_default_na: false
    urlpath: inventory.csv
  description: Describes the data in the hrrrzarr source
  driver: intake.source.csv.CSVSource
  metadata: {}


This source is almost how we want it, but the `urlpath` will not work after we push our catalog to Github. Intake sets the `CATALOG_DIR` parameter to point to whatever directory the catalog file is in. Using this parameter we can generate a url that will work even after we push the repository to Github.

In [11]:
sources['data_dictionary'] = yaml.load(source.yaml(), 
                                       Loader=yaml.CLoader)['sources']['data_dictionary']
data_dictionary_args = sources['data_dictionary']['args']
data_dictionary_args['urlpath'] = "{{ CATALOG_DIR }}/inventory.csv"
print(yaml.dump(sources))

data_dictionary:
  args:
    csv_kwargs:
      keep_default_na: false
    urlpath: '{{ CATALOG_DIR }}/inventory.csv'
  description: Describes the data in the hrrrzarr source
  driver: intake.source.csv.CSVSource
  metadata: {}
hrrrzarr:
  args:
    chunks:
      projection_x_coordinate: 1799
      projection_y_coordinate: 1059
    storage_options:
      anon: true
    urlpath:
    - s3://hrrrzarr/sfc/{{date.strftime('%Y%m%d/%Y%m%d_%Hz_anl.zarr')}}/{{level}}/{{param}}
    - s3://hrrrzarr/sfc/{{date.strftime('%Y%m%d/%Y%m%d_%Hz_anl.zarr')}}/{{level}}/{{param}}/{{level}}
  description: Mesowest's HRRR data. See readme source for more information.
  driver: intake_xarray.xzarr.ZarrSource
  metadata:
    coords: !!python/tuple
    - projection_x_coordinate
    - projection_y_coordinate
    dims:
      projection_x_coordinate: 1799
      projection_y_coordinate: 1059
  parameters:
    date:
      default: '2016-08-24T00:00:00'
      description: Date and hour of data.
      min: '2016-08-24T0

Your source now points to a inventory.csv file in the same directory as your catalog. Be sure to copy the file into your "intake_demo" directory.

Now that we have a source and a data dictionary to describe it lets add a readme to our catalog to explain how to use it and give some usage examples. This readme will also be displayed as the readme for the repository on Github. In this directory there is an example readme markdown file to use. Go ahead and copy it into the "intake_demo" directory.

In [12]:
source = intake.open_markdown('README.md')
source.name = 'readme'
source.description = 'Learn more about how to use this catalog'
source.read()

We will change the urlpath of this source in the same way as the data dictionary to ensure the readme loads correctly.

In [13]:
sources['readme'] = yaml.load(source.yaml(), Loader=yaml.CLoader)['sources']['readme']
readme_args = sources['readme']['args']
readme_args['urlpath'] = "{{ CATALOG_DIR }}/README.md"
print(yaml.dump(source_dict))

sources:
  data_dictionary:
    args:
      csv_kwargs:
        keep_default_na: false
      urlpath: '{{ CATALOG_DIR }}/inventory.csv'
    description: Describes the data in the hrrrzarr source
    driver: intake.source.csv.CSVSource
    metadata: {}
  hrrrzarr:
    args:
      chunks:
        projection_x_coordinate: 1799
        projection_y_coordinate: 1059
      storage_options:
        anon: true
      urlpath:
      - s3://hrrrzarr/sfc/{{date.strftime('%Y%m%d/%Y%m%d_%Hz_anl.zarr')}}/{{level}}/{{param}}
      - s3://hrrrzarr/sfc/{{date.strftime('%Y%m%d/%Y%m%d_%Hz_anl.zarr')}}/{{level}}/{{param}}/{{level}}
    description: Mesowest's HRRR data. See readme source for more information.
    driver: intake_xarray.xzarr.ZarrSource
    metadata:
      coords: !!python/tuple
      - projection_x_coordinate
      - projection_y_coordinate
      dims:
        projection_x_coordinate: 1799
        projection_y_coordinate: 1059
    parameters:
      date:
        default: '2016-08-24T00:00:0

With all our sources made, we will add them to the catalog, and save the catalog.

In [14]:
catalog['sources'] = sources
with open('intake_demo/catalog.yml', 'w') as f:
    yaml.dump(catalog, f)

At this point you should have three files in your "intake_demo" directory: "catalog.yml", "inventory.csv", and "README.md". All we need to do now is commit our changes and push them to Github.

```bash
git add .
git commit -m "initial commit"
git push
```

## Testing the Catalog

Now that your catalog is on Github let's try using it. In the cell below replace `jnmorley` with your own github username.

In [15]:
cat = intake.open_catalog('https://raw.githubusercontent.com/jnmorley/intake_demo/main/catalog.yml')
list(cat)

['data_dictionary', 'hrrrzarr', 'readme']

In [16]:
cat.readme.read()

In [17]:
cat.data_dictionary.read()

,Parameter Long Name,Vertical Level,Parameter Short Name,Units,1st Version Available,Analysis or Forcast,Notes
0,Hail,0.1_sigma_layer,HAIL_max_fcst,m,V4,anl,
1,Hail,0.1_sigma_layer,HAIL_1hr_max_fcst,m,V4,fcst,
2,Vertical Velocity,0.5_0.8_sigma_layer,DZDT_ave_fcst,m/s,V3,anl,
3,Vertical Velocity,0.5_0.8_sigma_layer,DZDT_1hr_ave_fcst,m/s,V3,fcst,
4,U-Component Vertical Wind Shear,0_1000m_above_ground,VUCSH,1/s,V3,both,
...,...,...,...,...,...,...,...
187,Upward Short-Wave Rad. Flux,top_of_atmosphere,USWRF,W/m2,V3,both,
188,"Simulated Brightness Temperature for GOES 11, ...",top_of_atmosphere,SBT113,K,V3,both,
189,"Simulated Brightness Temperature for GOES 11, ...",top_of_atmosphere,SBT114,K,V3,both,
190,"Simulated Brightness Temperature for GOES 12, ...",top_of_atmosphere,SBT123,K,V3,both,


In [18]:
cat.hrrrzarr.read()

<xarray.Dataset>
Dimensions:                  (projection_x_coordinate: 1799,
                              projection_y_coordinate: 1059)
Coordinates:
  * projection_x_coordinate  (projection_x_coordinate) float64 -2.698e+06 ......
  * projection_y_coordinate  (projection_y_coordinate) float64 -1.587e+06 ......
Data variables:
    TMP                      (projection_y_coordinate, projection_x_coordinate) float16 ...
    forecast_period          timedelta64[ns] 00:00:00
    forecast_reference_time  datetime64[ns] 2016-08-24
    height                   float64 1e+03
    pressure                 float64 2.5e+04
    time                     datetime64[ns] 2016-08-24

---

## Summary
In this tutorial we learned to create Intake catalogs and host them on Github. We learned to create sources with Intake and then modify them to make them more general. We explored a possible method for documenting data by adding a readme and data dictionary to our catalog. These guidelines will help you make your data more accessible to collaborators. 

## Resources and references
- [Intake Documentation](https://intake.readthedocs.io/en/latest/)
- [NOAA High-Resolution Rapid Refresh (HRRR) Data Archive](https://mesowest.utah.edu/html/hrrr/)
- [HRRR Zarr Variable List](https://mesowest.utah.edu/html/hrrr/zarr_documentation/html/zarr_variables.html)
- [AWS hrrrzarr Bucket](https://hrrrzarr.s3.amazonaws.com/index.html)